In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = saw
band = [3]
commitnumber = de5d43e
molecule = {'n2o': 3.2e-07}
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o n2o band3a
  atmpro = mls
  band = 3a
  commitnumber = a06b618
  conc = 3.2e-07
  dv = 0.001
  klin = 2.22e-20
  molecule = n2o
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 620
  vmin = 540
  w_diffuse = [(1.8,), (1.66, 1.8)]
  wgt = [(0.9,), (0.5, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.009795                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.011185
0.000750      2.0  0.010701                  2.0  0.011892
0.001052      3.0  0.012586                  3.0  0.013451
0.001476      4.0  0.014567                  4.0  0.015052
0.002070      5.0  0.016622                  5.0  0.016711
0.002904      6.0  0.018753                  6.0  0.018432
0.004074      7.0  0.020958                  7.0  0.020206
0.005714      8.0  0.023231                  8.0  0.022039
0.008015      9.0  0.025583                  9.0  0.023934
0.011243     10.0  0.028015                 10.0  0.025895
0.015771     11.0  0.030517                 11.0  0.027919
0.022122     12.0  0.033081                 12.0  0.030007
0.031031     13.0  0.035697                 13.0  0.032152
0.043528     14.0  0.038046                 14.0  0.034104
0.061057     15.0  0.039845                 15.0  0.035644
0.085645     16.0  0.041380                 16.0  0.036999
0.120136     17.0  0.042881                 17.0  0.038375
0.168516     18.0  0.044359                 18.0  0.039779
0.236378     19.0  0.045802                 19.0  0.041206
0.331549     20.0  0.047221                 20.0  0.042638
0.465100     21.0  0.048434                 21.0  0.043939
0.652400     22.0  0.046684                 22.0  0.042874
0.915100     23.0  0.041124                 23.0  0.038719
1.283650     24.0  0.034734                 24.0  0.033829
1.800600     25.0  0.028815                 25.0  0.029168
2.525700     26.0  0.023412                 26.0  0.024703
3.542800     27.0  0.018559                 27.0  0.020432
4.969550     28.0  0.014772                 28.0  0.016810
6.970850     29.0  0.012321                 29.0  0.014074
9.778100     30.0  0.010514                 30.0  0.011721
13.715850    31.0  0.008844                 31.0  0.009391
19.239350    32.0  0.007297                 32.0  0.007197
26.987250    33.0  0.006484                 33.0  0.005713
37.855300    34.0  0.006416                 34.0  0.005001
53.100050    35.0  0.006318                 35.0  0.004504
73.887500    36.0  0.006011                 36.0  0.004116
97.662500    37.0  0.005519                 37.0  0.003788
121.437500   38.0  0.004772                 38.0  0.003345
145.212500   39.0  0.003915                 39.0  0.002825
168.987500   40.0  0.003141                 40.0  0.002351
192.762500   41.0  0.002427                 41.0  0.001908
216.537500   42.0  0.001748                 42.0  0.001482
240.312500   43.0  0.001082                 43.0  0.001066
264.087500   44.0  0.000410                 44.0  0.000660
287.862500   45.0 -0.000193                 45.0  0.000338
311.637500   46.0 -0.000089                 46.0  0.000592
335.412500   47.0  0.001004                 47.0  0.001637
359.187500   48.0  0.002445                 48.0  0.002991
382.962500   49.0  0.003837                 49.0  0.004340
406.737500   50.0  0.005167                 50.0  0.005659
430.512500   51.0  0.006448                 51.0  0.006945
454.287500   52.0  0.007685                 52.0  0.008199
478.062500   53.0  0.008883                 53.0  0.009422
501.837500   54.0  0.010042                 54.0  0.010609
525.612500   55.0  0.011158                 55.0  0.011746
549.387500   56.0  0.012227                 56.0  0.012836
573.162500   57.0  0.013240                 57.0  0.013873
596.937500   58.0  0.014190                 58.0  0.014848
620.712500   59.0  0.015070                 59.0  0.015754
644.487500   60.0  0.015875                 60.0  0.016585
668.262500   61.0  0.016602                 61.0  0.017340
692.037500   62.0  0.017260                 62.0  0.018027
715.812500   63.0  0.017857                 63.0  0.018652
739.587500   64.0  0.018404                 64.0  0.019222
76

# Fluxes by Layer

CRD                          CLIRAD            \
                         flug      fldg      fnetg       flug      fldg   
pressure     level                                                        
0.000000e+00 1     -23.030142  0.000000 -23.030142        NaN       NaN   
1.000000e-08 1            NaN       NaN        NaN -23.050441  0.000002   
6.244000e-04 2     -23.030143  0.000001 -23.030141 -23.050441  0.000003   
8.759000e-04 3     -23.030143  0.000002 -23.030141 -23.050442  0.000003   
1.228600e-03 4     -23.030143  0.000003 -23.030141 -23.050442  0.000004   
1.723400e-03 5     -23.030144  0.000004 -23.030140 -23.050442  0.000005   
2.417400e-03 6     -23.030144  0.000006 -23.030138 -23.050442  0.000007   
3.390900e-03 7     -23.030145  0.000008 -23.030136 -23.050442  0.000009   
4.756500e-03 8     -23.030145  0.000012 -23.030133 -23.050442  0.000012   
6.672000e-03 9     -23.030145  0.000018 -23.030127 -23.050442  0.000017   
9.358900e-03 10    -23.030146  0.000026 -23.030119 -23.050441  0.000024   
1.312780e-02 11    -23.030146  0.000039 -23.030107 -23.050439  0.000034   
1.841450e-02 12    -23.030144  0.000057 -23.030088 -23.050437  0.000049   
2.583020e-02 13    -23.030142  0.000083 -23.030059 -23.050432  0.000070   
3.623230e-02 14    -23.030136  0.000122 -23.030015 -23.050424  0.000102   
5.082340e-02 15    -23.030127  0.000178 -23.029949 -23.050412  0.000149   
7.129060e-02 16    -23.030111  0.000259 -23.029852 -23.050392  0.000216   
1.000000e-01 17    -23.030086  0.000374 -23.029712 -23.050363  0.000312   
1.402710e-01 18    -23.030046  0.000539 -23.029507 -23.050318  0.000450   
1.967600e-01 19    -23.029984  0.000774 -23.029210 -23.050250  0.000649   
2.759970e-01 20    -23.029888  0.001108 -23.028780 -23.050149  0.000934   
3.871000e-01 21    -23.029740  0.001581 -23.028159 -23.049997  0.001343   
5.431000e-01 22    -23.029512  0.002248 -23.027264 -23.049772  0.001930   
7.617000e-01 23    -23.029201  0.003146 -23.026055 -23.049468  0.002737   
1.068500e+00 24    -23.028839  0.004279 -23.024560 -23.049116  0.003792   
1.498800e+00 25    -23.028458  0.005668 -23.022790 -23.048743  0.005144   
2.102400e+00 26    -23.028086  0.007357 -23.020730 -23.048381  0.006867   
2.949000e+00 27    -23.027777  0.009396 -23.018382 -23.048085  0.009049   
4.136600e+00 28    -23.027628  0.011857 -23.015771 -23.047953  0.011792   
5.802500e+00 29    -23.027753  0.014897 -23.012856 -23.048098  0.015254   
8.139200e+00 30    -23.028266  0.018821 -23.009445 -23.048638  0.019690   
1.141700e+01 31    -23.029386  0.024023 -23.005363 -23.049790  0.025394   
1.601470e+01 32    -23.031530  0.030984 -23.000546 -23.051952  0.032670   
2.246400e+01 33    -23.035346  0.040375 -22.994971 -23.055716  0.041934   
3.151050e+01 34    -23.041459  0.053436 -22.988023 -23.061705  0.054045   
4.420010e+01 35    -23.050461  0.072084 -22.978378 -23.070618  0.070476   
6.200000e+01 36    -23.063536  0.098480 -22.965057 -23.083777  0.093133   
8.577500e+01 37    -23.081657  0.133529 -22.948128 -23.102312  0.123262   
1.095500e+02 38    -23.100537  0.167953 -22.932583 -23.121840  0.153461   
1.333250e+02 39    -23.120497  0.201354 -22.919143 -23.142536  0.183577   
1.571000e+02 40    -23.141751  0.233635 -22.908117 -23.164507  0.213506   
1.808750e+02 41    -23.164264  0.264993 -22.899270 -23.187678  0.243300   
2.046500e+02 42    -23.188023  0.295587 -22.892436 -23.211995  0.272991   
2.284250e+02 43    -23.213053  0.325541 -22.887513 -23.237422  0.302591   
2.522000e+02 44    -23.239414  0.354949 -22.884464 -23.263928  0.332101   
2.759750e+02 45    -23.267207  0.383896 -22.883311 -23.291484  0.361515   
2.997500e+02 46    -23.296446  0.412593 -22.883853 -23.319957  0.390940   
3.235250e+02 47    -23.326221  0.442118 -22.884103 -23.348526  0.421176   
3.473000e+02 48    -23.355098  0.473823 -22.881274 -23.376025  0.453287   
3.710750e+02 49    -23.382444  0.508056 -22.874388 -23.401954  0.487640   
3.948500e+02 50    -23.408174  0.544592 -22.863581 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')